#  Generating GRCh37 Medical Genes Benchmark
This notebook details the steps to generate the challenging medically relevant genes benchmark. All paths are from the top level directory of the repository and large dependency files. External data dependencies are downloaded from the GIAB FTP site as needed.

## Steps  

1. Look up coordinates for gene symbols in ENSEMBLE GRCh37 Human Genes v101 of the union of Mandelker et al Supplementary Table 13, COSMIC Cancer Gene Census, Steve Lincoln Medical Gene Lists -- `scripts\GRCh37_lookup_MRG_symbol_coordinates_ENSEMBL.R`

2. Find overlap of genes with HG002 v4.2.1, then add slop and find overlap with HG002 trio-hifiasm (v0.11) diploid assembly confident regions `HG002v11-align2-GRCh37.dip.bed`

3. Find genes with < 90% included bases in v4.2.1 HG002 small variant benchmark and fully covered with overlapping segdups and flanking sequence in HG002 hifiasm v0.11 GRCh37 dip.bed, find union of GRCh37 and GRCh38 MRG lists, then add genes that are unique to GRCh37 but still fully covered with overlapping segdups and flanking sequence in HG002 hifiasm v0.11 GRCh37 dip.bed -- `scripts/find_coordinates_of_MRG_GRCh37_GRCh38_union.R`

4. Use coordinates for benchmark then remove
    - homopolymers and imperfect homopolymers > 20
    - SVs with 50bp flanking and overlapping tandem repeats
    - hifiasm error
    - GRCh37 GAPs
    - Remove partially covered tandem repeats
    - Remove MHC region
    
5. Generate stratification files for Complex Variants in Tandem Repeats
    - GRCh37_MRG_stratification_ComplexVar_in_TR.bed

# Downloading Data Dependencies

## hifiasm Variants and Diploid Regions

In [1]:
mkdir -p data/hifiasm_dipcall_output
wget -O data/hifiasm_dipcall_output/HG002v11-align2-GRCh37.dip.bed \
    https://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/data/AshkenazimTrio/analysis/NIST_HG002_medical_genes_benchmark_v0.02/GRCh37/hifiasm_v0.11/HG002v11-align2-GRCh37.dip.bed
wget -O data/hifiasm_dipcall_output/HG002v11-align2-GRCh37.dip.vcf.gz \
    https://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/release/AshkenazimTrio/HG002_NA24385_son/CMRG_v1.00/GRCh37/SupplementaryFiles/HG002v11-align2-GRCh37/HG002v11-align2-GRCh37.dip.vcf.gz

--2021-08-31 12:22:05--  https://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/data/AshkenazimTrio/analysis/NIST_HG002_medical_genes_benchmark_v0.02/GRCh37/hifiasm_v0.11/HG002v11-align2-GRCh37.dip.bed
Resolving ftp-trace.ncbi.nlm.nih.gov (ftp-trace.ncbi.nlm.nih.gov)... 2607:f220:41e:250::12, 2607:f220:41e:250::10, 165.112.9.229, ...
Connecting to ftp-trace.ncbi.nlm.nih.gov (ftp-trace.ncbi.nlm.nih.gov)|2607:f220:41e:250::12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20530 (20K) [application/vnd.realvnc.bed]
Saving to: ‘data/hifiasm_dipcall_output/HG002v11-align2-GRCh37.dip.bed’

data/hifiasm_dipcal 100%[===================>]  20.05K  --.-KB/s    in 0.02s   

2021-08-31 12:22:05 (1.25 MB/s) - ‘data/hifiasm_dipcall_output/HG002v11-align2-GRCh37.dip.bed’ saved [20530/20530]



: 1

## Reference Genome

In [ ]:
wget -O resources/hs37d5.fa.gz \
    https://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/release/references/GRCh37/hs37d5.fa.gz
gunzip resources/hs37d5.fa.gz
samtools faidx resources/hs37d5.fa

## Genomic Stratifications

In [ ]:
mkdir -p resources/giab_stratifications
wget -O resources/giab_stratifications/GRCh37_segdups.bed.gz \
    https://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/release/genome-stratifications/v2.0/GRCh37/SegmentalDuplications/GRCh37_segdups.bed.gz

wget -O resources/giab_stratifications/GRCh37_MHC.bed.gz \
    https://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/release/genome-stratifications/v2.0/GRCh37/OtherDifficult/GRCh37_MHC.bed.gz
    
wget -O resources/giab_stratifications/GRCh37_AllTandemRepeatsandHomopolymers_slop5.bed.gz \
    ftp://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/release/genome-stratifications/v2.0/GRCh37/LowComplexity/GRCh37_AllTandemRepeatsandHomopolymers_slop5.bed.gz

wget -O resources/giab_stratifications/GRCh37_AllTandemRepeats_gt100bp_slop5.bed \
    https://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/release/genome-stratifications/v2.0/GRCh37/LowComplexity/GRCh37_AllTandemRepeats_gt100bp_slop5.bed.gz

## GIAB Benchmark Sets

### CMRG Draft Benchmarks

In [ ]:
mkdir -p data/manually_created_files/cmrg_draft_benchmarks

## v0.02.03 small variant benchmark
wget -O data/manually_created_files/cmrg_draft_benchmarks/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark_v0.02.03.bed  \
    https://ftp-trace.ncbi.nlm.nih.gov/giab/ftp/data/AshkenazimTrio/analysis/NIST_HG002_medical_genes_benchmark_v0.02/GRCh37/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark_v0.02.03.bed
wget -O data/manually_created_files/cmrg_draft_benchmarks/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark_v0.02.03.vcf.gz  \
    https://ftp-trace.ncbi.nlm.nih.gov/giab/ftp/data/AshkenazimTrio/analysis/NIST_HG002_medical_genes_benchmark_v0.02/GRCh37/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark_v0.02.03.vcf.gz
wget -O data/manually_created_files/cmrg_draft_benchmarks/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark_v0.02.03.vcf.gz.tbi  \
    https://ftp-trace.ncbi.nlm.nih.gov/giab/ftp/data/AshkenazimTrio/analysis/NIST_HG002_medical_genes_benchmark_v0.02/GRCh37/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark_v0.02.03.vcf.gz.tbi
    

## v0.01 SV benchmark
wget -O data/manually_created_files/cmrg_draft_benchmarks/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.01.bed \
    https://ftp-trace.ncbi.nlm.nih.gov/giab/ftp/data/AshkenazimTrio/analysis/NIST_HG002_medical_genes_SV_benchmark_v0.01/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.01.bed
wget -O data/manually_created_files/cmrg_draft_benchmarks/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.01.vcf.gz \
    https://ftp-trace.ncbi.nlm.nih.gov/giab/ftp/data/AshkenazimTrio/analysis/NIST_HG002_medical_genes_SV_benchmark_v0.01/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.01.vcf.gz
wget -O data/manually_created_files/cmrg_draft_benchmarks/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.01.vcf.gz.tbi \
    https://ftp-trace.ncbi.nlm.nih.gov/giab/ftp/data/AshkenazimTrio/analysis/NIST_HG002_medical_genes_SV_benchmark_v0.01/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.01.vcf.gz.tbi


### V4.2.1 GRCh37 Benchmark

In [ ]:
mkdir -p data/v4.2.1_benchmark_regions
wget -O data/v4.2.1_benchmark_regions/HG002_GRCh37_1_22_v4.2.1_benchmark_noinconsistent.bed \
    https://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/release/AshkenazimTrio/HG002_NA24385_son/NISTv4.2.1/GRCh37/HG002_GRCh37_1_22_v4.2.1_benchmark_noinconsistent.bed

# From these genes to be benchmarked remove the following regions that we exclude from the diploid assembly based variant calls:

    - homopolymers and imperfect homopolymers > 20
    - SVs with 50bp flanking and overlapping tandem repeats
    - hifiasm error
    - GRCh37 GAPs
    - Remove partially covered tandem repeats
    
    

## Remove homopolymers > 20bp

In [ ]:
bedtools subtract \
    -a workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_CMRG_coordinates_temp_bedtools_merge.bed \
    -b data/giab_stratifications/GRCh37/GRCh37_SimpleRepeat_homopolymer_gt20_slop5.bed.gz \
    > workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_CMRG_coordinates_temp_subtract_GRCh37_SimpleRepeat_homopolymer_gt20_slop5.bed

## Remove imperfect homopolymers > 20bp

In [ ]:
bedtools subtract \
    -a workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_CMRG_coordinates_temp_subtract_GRCh37_SimpleRepeat_homopolymer_gt20_slop5.bed \
    -b data/giab_stratifications/GRCh37/GRCh37_SimpleRepeat_imperfecthomopolgt20_slop5.bed.gz \
    > workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_CMRG_coordinates_temp_subtract_GRCh37_SimpleRepeat_imperfecthomopolgt20_slop5.bed

## SVs with 50bp flanking and overlapping tandem repeats
`GRCh37_AllTandemRepeatsandHomopolymers_slop5.bed.gz` is from ftp://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/release/genome-stratifications/v2.0/GRCh38/LowComplexity/GRCh37_AllTandemRepeatsandHomopolymers_slop5.bed.gz

In [ ]:
gunzip \
    -c data/hifiasm_dipcall_output/HG002v11-align2-GRCh37.dip.vcf.gz \
    | awk 'length($4)>49 || length($5)>49' \
    | awk '{FS=OFS="\t"} {print $1,$2-1,$2+length($4)}' \
    > workflow/smallvar_benchmark/GRCh37/HG002v11-align2-GRCh37.dip_SVsgt49bp.bed

In [ ]:
intersectBed \
    -wa -a resources/giab_stratifications/GRCh37_AllTandemRepeatsandHomopolymers_slop5.bed.gz \
    -b workflow/smallvar_benchmark/GRCh37/HG002v11-align2-GRCh37.dip_SVsgt49bp.bed \
    | multiIntersectBed -i stdin workflow/smallvar_benchmark/GRCh37/HG002v11-align2-GRCh37.dip_SVsgt49bp.bed \
    |  awk '{FS=OFS="\t"} {print $1,$2-50,$3+50}' \
    | mergeBed -i stdin -d 1000 \
    > workflow/smallvar_benchmark/GRCh37/HG002v11-align2-GRCh37.dip_SVsgt49bp_repeatexpanded_slop50_merge1000.bed

In [ ]:
bedtools subtract \
    -a workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_CMRG_coordinates_temp_subtract_GRCh37_SimpleRepeat_imperfecthomopolgt20_slop5.bed \
    -b workflow/smallvar_benchmark/GRCh37/HG002v11-align2-GRCh37.dip_SVsgt49bp_repeatexpanded_slop50_merge1000.bed \
    > workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_CMRG_coordinates_temp_subtract_SVsgt49bp_repeatexpanded_slop50_merge1000.bed

## Remove hifiasm error on chr21
`GRCh37_hifiasm_error.bed`
 was created through manual curation of clusters of errors identified during evaluation steps of benchmark development

In [ ]:
bedtools subtract \
    -a workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_CMRG_coordinates_temp_subtract_SVsgt49bp_repeatexpanded_slop50_merge1000.bed \
    -b data/manually_created_files/GRCh37_hifiasm_error.bed \
    > workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_CMRG_coordinates_temp_subtract_hifiasm_error.bed

## Remove GRCh37 GAPs
`GRCh37_MRG_GAPs.bed` was created through manual curation of clusters of errors identified during evaluation steps of benchmark development


In [ ]:
bedtools subtract \
    -a workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_CMRG_coordinates_temp_subtract_hifiasm_error.bed \
    -b data/manually_created_files/GRCh37_MRG_GAPs.bed \
    > workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_CMRG_coordinates_temp_subtract_GRCh37_MRG_GAPs.bed

## Sort

In [ ]:
cat workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_CMRG_coordinates_temp_subtract_GRCh37_MRG_GAPs.bed \
    | sort -k1,1n -k2,2n \
    > workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_CMRG_coordinates_temp_subtract_GRCh37_MRG_GAPs_sorted.bed

## Remove partially covered tandem repeats

In [ ]:
complementBed \
    -i workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_CMRG_coordinates_temp_subtract_GRCh37_MRG_GAPs_sorted.bed \
    -g resources/human.b37.genome \
    | intersectBed \
        -wa -a resources/giab_stratifications/GRCh37_AllTandemRepeatsandHomopolymers_slop5.bed.gz \
        -b stdin \
    | subtractBed \
        -a workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_CMRG_coordinates_temp_subtract_GRCh37_MRG_GAPs_sorted.bed \
        -b stdin \
    > workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_CMRG_coordinates_temp_subtract_partial_tandem_repeats.bed

## Prepare Small Variant benchmark VCF

__Notes__
1. `workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark.vcf.gz` is  https://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/data/AshkenazimTrio/analysis/NIST_HG002_medical_genes_benchmark_v0.02/GRCh37/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark_v0.02.03.vcf.gz after updates to the headers

2. `workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark.bed` is https://ftp-trace.ncbi.nlm.nih.gov/ReferenceSamples/giab/data/AshkenazimTrio/analysis/NIST_HG002_medical_genes_benchmark_v0.02/GRCh37/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark_v0.02.03.bed

In [ ]:
cp workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_CMRG_coordinates_temp_subtract_partial_tandem_repeats.bed \
    workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark.bed

cat workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark.bed \
    | awk '{sum+=$3-$2} END {print sum}'

bedtools intersect \
    -a data/hifiasm_dipcall_output/HG002v11-align2-GRCh37.dip.vcf.gz \
    -b workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_CMRG_coordinates_temp_bedtools_merge.bed \
    -header \
    > workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark.vcf

bgzip -f workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark.vcf

tabix -f workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark.vcf.gz

## SV Benchmark

In [ ]:
# Find SVs MRG benchmark gene coordinates
mkdir -p workflow/SV_benchmark/GRCh37/
bedtools intersect \
    -a workflow/smallvar_benchmark/GRCh37//HG002v11-align2-GRCh37.dip_SVsgt49bp_repeatexpanded_slop50_merge1000.bed \
    -b workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_CMRG_coordinates_temp_bedtools_merge.bed \
    > workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37.dip_SVsgt49bp_repeatexpanded_slop50_merge1000_intersect_MRG_benchmark_coordinates.bed

In [ ]:
# Find SVs MRG benchmark gene coordinates
bedtools intersect \
    -a workflow/smallvar_benchmark/GRCh37//HG002v11-align2-GRCh37.dip_SVsgt49bp_repeatexpanded_slop50_merge1000.bed \
    -b workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_CMRG_coordinates_temp_bedtools_merge.bed \
    > workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37.dip_SVsgt49bp_repeatexpanded_slop50_merge1000_intersect_MRG_benchmark_coordinates.bed

# Subset to SVs only gt49bp 
gunzip \
    -c data/hifiasm_dipcall_output/HG002v11-align2-GRCh37.dip.vcf.gz \
    | awk '{FS="\t|,"} {if($1 ~ /^#/ || length($4)-length($5)>49 || length($5)-length($4)>49 || length($6)-length($4)>49) print}' \
    | intersectBed \
        -a workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37.dip_SVsgt49bp_repeatexpanded_slop50_merge1000_intersect_MRG_benchmark_coordinates.bed \
        -b stdin -c \
    | awk '$4>0' \
    | cut -f1-3 \
    > workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37.dip_SVsgt49bp_repeatexpanded_slop50_merge1000_intersect_MRG_benchmark_coordinates_onlygt49bp.bed 

# Find isolated SVs  

gunzip \
    -c data/hifiasm_dipcall_output/HG002v11-align2-GRCh37.dip.vcf.gz \
    | awk '{FS="\t|,"} {if($1 ~ /^#/ || length($4)-length($5)>9 || length($5)-length($4)>9 || length($6)-length($4)>9) print}' \
    | intersectBed -a workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37.dip_SVsgt49bp_repeatexpanded_slop50_merge1000_intersect_MRG_benchmark_coordinates_onlygt49bp.bed \
    -b stdin -c \
    | awk '$4==1' \
    | cut -f1-3 \
    > workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37.dip_SVsgt49bp_repeatexpanded_slop50_merge1000_intersect_MRG_benchmark_coordinates_onlygt49bp_isolated.bed 

# Find complex SVs  

gunzip \
    -c data/hifiasm_dipcall_output/HG002v11-align2-GRCh37.dip.vcf.gz \
    | awk '{FS="\t|,"} {if($1 ~ /^#/ || length($4)-length($5)>9 || length($5)-length($4)>9 || length($6)-length($4)>9) print}' \
    | intersectBed \
        -a workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37.dip_SVsgt49bp_repeatexpanded_slop50_merge1000_intersect_MRG_benchmark_coordinates_onlygt49bp.bed \
        -b stdin -c \
        | awk '$4>1' \
        | cut -f1-3 \
        > workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37.dip_SVsgt49bp_repeatexpanded_slop50_merge1000_intersect_MRG_benchmark_coordinates_onlygt49bp_complexSVs.bed 

# Remove complex SVs from MRG gene candidate coordinates and remove GAPs 
bedtools subtract \
    -a workflow/smallvar_benchmark/GRCh37/HG002_GRCh37_CMRG_coordinates_temp_bedtools_merge.bed \
    -b workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37.dip_SVsgt49bp_repeatexpanded_slop50_merge1000_intersect_MRG_benchmark_coordinates_onlygt49bp_complexSVs.bed \
    | bedtools subtract \
        -a stdin \
        -b  data/manually_created_files/GRCh37_MRG_GAPs.bed \
    > workflow/SV_benchmark/GRCh37/HG002_GRCh37_MRG_draft_SV_benchmark_temp.bed

#HG002v11-align2-GRCh37.dip_complexindelsgt9bpinRepeats.bed  from the SV benchmark bed:
# Find tandem repeats and homopolymers that have multiple indels >9bp, since these can add up to >49bp and should be subtracted from the benchmark SV bed
gunzip \
    -c data/hifiasm_dipcall_output/HG002v11-align2-GRCh37.dip.vcf.gz \
    | awk '{FS="\t|,"} {if($1 ~ /^#/ || length($4)-length($5)>9 || length($5)-length($4)>9 || length($6)-length($4)>9) print}' \
    | intersectBed \
        -a resources/giab_stratifications/GRCh37_AllTandemRepeatsandHomopolymers_slop5.bed.gz \
        -b stdin -c \
    | awk '$4>1' \
    | cut -f1-3 \
    | intersectBed \
        -v -a stdin \
        -b workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37.dip_SVsgt49bp_repeatexpanded_slop50_merge1000_intersect_MRG_benchmark_coordinates_onlygt49bp.bed \
        > workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37.dip_complexindelsgt9bpinRepeats.bed

bedtools subtract \
    -a workflow/SV_benchmark/GRCh37/HG002_GRCh37_MRG_draft_SV_benchmark_temp.bed \
    -b workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37.dip_complexindelsgt9bpinRepeats.bed \
    > workflow/SV_benchmark/GRCh37/HG002_GRCh37_MRG_draft_SV_benchmark.bed

cat workflow/SV_benchmark/GRCh37/HG002_GRCh37_MRG_draft_SV_benchmark.bed \
    | awk '{sum+=$3-$2} END {print sum}'

# Decompose for truvari comparison
#vt decompose -s HG002v11-align2-GRCh37.dip.vcf -o HG002v11-align2-GRCh37.dip_decomposed.vcf
#python script to remove ambiguous (non-ACTGN) REF

gunzip \
    -c data/hifiasm_dipcall_output/HG002v11-align2-GRCh37.dip.vcf.gz \
    > data/hifiasm_dipcall_output/HG002v11-align2-GRCh37.dip.vcf


## remove ambiguous (non-ACGTN) in REF field. Adjust path to where you keep this file
python scripts/fix_reference_allele.py \
    --input_vcf_file data/hifiasm_dipcall_output/HG002v11-align2-GRCh37.dip.vcf \
    --output_file workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37_noambig.dip.vcf

rm data/hifiasm_dipcall_output/HG002v11-align2-GRCh37.dip.vcf

## zip for bcftools
bgzip \
    -c workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37_noambig.dip.vcf \
    > workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37_noambig.dip.vcf.gz

tabix workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37_noambig.dip.vcf.gz

In [ ]:
## split multiallelic to biallelic
bcftools norm -m- \
    -Oz workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37_noambig.dip.vcf.gz \
    -o workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37_noambig_norm_m.vcf.gz

## left align and normalize indels. 
bcftools norm \
    -f resources/hs37d5.fa \
    -Oz -o workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37_noambig_norm_mf.vcf.gz \
    workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37_noambig_norm_m.vcf.gz

## remove duplicate records
bcftools norm -d none \
    -Oz workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37_noambig_norm_mf.vcf.gz \
    -o workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37_noambig_norm_mfd.vcf.gz

In [ ]:
## remove MHC region
bedtools subtract \
    -header \
    -a workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37_noambig_norm_mfd.vcf.gz \
    -b resources/giab_stratifications/GRCh37_MHC.bed.gz \
    | bgzip -c \
    > workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37_noambig_norm_mfd_noMHC.vcf.gz

In [ ]:
## intersect w/ benchmark bed and subset to >39bp in REF or ALT fields. 
#intersect w/ MRG target regions and subset >39 bp

bedtools intersect -header \
    -a workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37_noambig_norm_mfd_noMHC.vcf.gz \
    -b workflow/SV_benchmark/GRCh37/HG002_GRCh37_MRG_draft_SV_benchmark.bed \
    | awk '$1 ~ /^#/ || length($4)>39 || length($5)>39' \
    | bgzip -c \
    > workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37_noambig_norm_mfd_noMHC_intersectBenchBED_gt39bp.vcf.gz

## index vcf, required by truvari
tabix -p vcf workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37_noambig_norm_mfd_noMHC_intersectBenchBED_gt39bp.vcf.gz

In [ ]:
## Find benchmark variants between 35 and 49 base pairs in size and exclude overlapping tandem repeats plus slop 50bp on either side. Remove these from the benchmark regions bed so that it includes only SVs that are greater than 49 base pairs
gunzip workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37_noambig_norm_mfd_noMHC_intersectBenchBED_gt39bp.vcf
python scripts/SVs_between_35_50bp.py \
    --input workflow/SV_benchmark/GRCh37/HG002v11-align2-GRCh37_noambig_norm_mfd_noMHC_intersectBenchBED_gt39bp.vcf \
    --output workflow/SV_benchmark/GRCh37/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.01.02_SVs_gt34_and_lt_50bp.vcF

In [ ]:
# vcf2bed from bedops
vcf2bed < workflow/SV_benchmark/GRCh37/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.01.02_SVs_gt34_and_lt_50bp.vcf \
    > workflow/SV_benchmark/GRCh37/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.01.02_SVs_gt34_and_lt_50bp.bed

In [ ]:
intersectBed -wa \
    -a resources/giab_stratifications/GRCh37_AllTandemRepeatsandHomopolymers_slop5.bed.gz \
    -b workflow/SV_benchmark/GRCh37/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.01.02_SVs_gt34_and_lt_50bp.bed \
    | multiIntersectBed \
        -i stdin workflow/SV_benchmark/GRCh37/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.01.02_SVs_gt34_and_lt_50bp.bed \
    |  awk '{FS=OFS="\t"} {print $1,$2-50,$3+50}' \
    > workflow/SV_benchmark/GRCh37/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.01.02_gt34_and_lt_50bp_repeatexpanded_slop50.bed

In [ ]:
bedtools subtract \
    -a workflow/SV_benchmark/GRCh37/HG002_GRCh37_MRG_draft_SV_benchmark.bed \
    -b workflow/SV_benchmark/GRCh37/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.01.02_gt34_and_lt_50bp_repeatexpanded_slop50.bed \
    > workflow/SV_benchmark/GRCh37/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.01.bed

## Prepare release benchmark files

Steps for creating excluding errors found in curation of the MRG benchmark, in order to create v1.0 bed files for small variants and SVs  

Updated 4/27/21 to exclude 50bp on either side of errors and unsure variants found in curation, because sometimes a deletion and overlapping SNVs outside a repeat weren't excluded from benchmark stats. Also, remove ".;" at beginning of INFO field introduced by svanalyzer svwiden. Produces v1.00.01 small variant and structural variant MRG benchmarks

__used NCBI remap to generate__ GRCh38_curation_medicalgene_SV_errorsorunsure_repeatexpanded.bed


In [ ]:
##errors to exclude found in curation of SVs
#only found 2 errors so manually created bed GRCh37_curation_medicalgene_SV_errorsorunsure.bed
#expand bed coordinates to completely cover any overlapping homopolymers and tandem repeats
mkdir -p workflow/release_benchmark_generation
intersectBed -wa \
    -a resources/giab_stratifications/GRCh37_AllTandemRepeatsandHomopolymers_slop5.bed.gz \
    -b data/manually_created_files/GRCh37_curation_medicalgene_SV_errorsorunsure.bed \
    | multiIntersectBed -i stdin data/manually_created_files/GRCh37_curation_medicalgene_SV_errorsorunsure.bed \
    | mergeBed -i stdin \
    > workflow/release_benchmark_generation/GRCh37_curation_medicalgene_SV_errorsorunsure_repeatexpanded.bed

In [ ]:
#create bed with sites curated as unsure or incorrect in the benchmark in GRCh37 coordinates
cut -f3,6,8,12,20 \
    data/manually_created_files/combined\ curation\ responses\ from\ benchmarking\ with\ sm\ variant\ v0.02.03\ -\ GRCh37andGRCh38.tsv \
    | grep 'sure\|o' \
    | grep -v ^ref \
    | awk '{FS=OFS="\t"} {print $2, $3-50, $3+length($4)+50}' \
    | sort -k1,1n -k2,2n -k3,3n \
    | mergeBed -i stdin -d 100 \
    > workflow/release_benchmark_generation/GRCh37_curation_medicalgene_smallvar_errorsorunsure.bed

intersectBed -wa \
    -a resources/giab_stratifications/GRCh37_AllTandemRepeatsandHomopolymers_slop5.bed.gz \
    -b workflow/release_benchmark_generation/GRCh37_curation_medicalgene_smallvar_errorsorunsure.bed \
    | multiIntersectBed \
        -i stdin workflow/release_benchmark_generation/GRCh37_curation_medicalgene_smallvar_errorsorunsure.bed \
    | mergeBed -i stdin \
    > workflow/release_benchmark_generation/GRCh37_curation_medicalgene_smallvar_errorsorunsure_repeatexpanded.bed

In [ ]:
cd data/manually_created_files/
##for manuscript, identify number of errors/unsure identified in curation and the number of these that would have been excluded by curating common FPs/FNs first
#create bed with sites curated as unsure or incorrect in the benchmark in GRCh37 coordinates
grep ^Common combined\ curation\ responses\ from\ benchmarking\ with\ sm\ variant\ v0.02.03\ -\ GRCh37andGRCh38.tsv \
    | cut -f3,6,8,12,20 \
    | grep 'sure\|o' \
    | grep -v ^ref \
    | awk '{FS=OFS="\t"} {print $2, $3-50, $3+length($4)+50}' \
    | sort -k1,1n -k2,2n -k3,3n \
    | mergeBed -i stdin -d 100 \
    > ../../workflow/release_benchmark_generation/GRCh37_curation_medicalgene_smallvar_errorsorunsure_Commononly.bed

grep -v ^Common combined\ curation\ responses\ from\ benchmarking\ with\ sm\ variant\ v0.02.03\ -\ GRCh37andGRCh38.tsv \
    | cut -f3,6,8,12,20 \
    | grep 'sure\|o' \
    | grep -v ^ref \
    | grep ^GRCh37 \
    | awk '{FS=OFS="\t"} {print $2, $3-50, $3+length($4)+50}' \
    | sort -k1,1n -k2,2n -k3,3n  \
    > ../../workflow/release_benchmark_generation/GRCh37_curation_medicalgene_smallvar_errorsorunsure_evaluationonly.bed

cd ../..

In [ ]:
## Sanity check - expected value 50
wc -l workflow/release_benchmark_generation/GRCh37_curation_medicalgene_smallvar_errorsorunsure_evaluationonly.bed

In [ ]:
#expand bed coordinates to completely cover any overlapping homopolymers and tandem repeats
## Sanity Check - expected value 6
intersectBed -wa \
    -a resources/giab_stratifications/GRCh37_AllTandemRepeatsandHomopolymers_slop5.bed.gz \
    -b workflow/release_benchmark_generation/GRCh37_curation_medicalgene_smallvar_errorsorunsure_Commononly.bed \
    | multiIntersectBed \
        -i stdin workflow/release_benchmark_generation/GRCh37_curation_medicalgene_smallvar_errorsorunsure_Commononly.bed \
    | mergeBed -i stdin \
    | intersectBed -v \
        -a workflow/release_benchmark_generation/GRCh37_curation_medicalgene_smallvar_errorsorunsure_evaluationonly.bed \
        -b stdin \
    | wc -l

exclude errors/unsure sites found in SV curation, small variant curation, small variant complex TR curation, and FPs from complex TR comparison


In [ ]:
subtractBed \
        -a data/manually_created_files/cmrg_draft_benchmarks/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark_v0.02.03.bed \
        -b workflow/release_benchmark_generation/GRCh37_curation_medicalgene_SV_errorsorunsure_repeatexpanded.bed \
    | subtractBed \
        -a stdin \
        -b workflow/release_benchmark_generation/GRCh37_curation_medicalgene_smallvar_errorsorunsure_repeatexpanded.bed \
    | subtractBed \
        -a stdin \
        -b data/manually_created_files/GRCh37_curation_medicalgene_smallvar_complexrepeat_errorsorunsure_repeatexpanded.bed \
    | subtractBed \
        -a stdin \
        -b data/manually_created_files/HiCanu_2.1_HG002_GRCh37_difficult_medical_gene_smallvar_benchmark_v0.02.03_intersected_FPs_repeatexpanded_slop50.bed \
        > workflow/release_benchmark_generation/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark_v1.00.01.bed

__Sanity Checks__ Make sure the bed size doesn't decrease more than expected

In [ ]:
## Sanity Check - expected value 11712171
awk '{sum+=$3-$2} END {print sum}' \
    data/manually_created_files/cmrg_draft_benchmarks/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark_v0.02.03.bed 

In [ ]:
## Sanity Check - expected value 11679803
awk '{sum+=$3-$2} END {print sum}' \
    workflow/release_benchmark_generation/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark_v1.00.01.bed 

Make sure the number of variants in the bed don't decrease more than expected

In [ ]:
## Sanity check - expected number of lines 21907
intersectBed -a data/manually_created_files/cmrg_draft_benchmarks/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark_v0.02.03.vcf.gz \
    -b data/manually_created_files/cmrg_draft_benchmarks/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark_v0.02.03.bed \
    | wc -l

In [ ]:
## Sanity check - expected number of lines 21591
intersectBed -a data/manually_created_files/cmrg_draft_benchmarks/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark_v0.02.03.vcf.gz \
    -b workflow/release_benchmark_generation/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark_v1.00.01.bed | wc -l


Using v0.02.03 vcf as v1.00.01 vcf

In [ ]:
cp data/manually_created_files/cmrg_draft_benchmarks/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark_v0.02.03.vcf.gz \
    workflow/release_benchmark_generation/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark_v1.00.01.vcf.gz 
cp data/manually_created_files/cmrg_draft_benchmarks/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark_v0.02.03.vcf.gz.tbi \
    workflow/release_benchmark_generation/HG002_GRCh37_difficult_medical_gene_smallvar_benchmark_v1.00.01.vcf.gz.tbi

Excluding errors/unsure sites found in SV curation

In [ ]:
subtractBed \
    -a data/manually_created_files/cmrg_draft_benchmarks/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.01.bed \
    -b workflow/release_benchmark_generation/GRCh37_curation_medicalgene_SV_errorsorunsure_repeatexpanded.bed \
    > workflow/release_benchmark_generation/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.02.00.bed

__Sanity Check__ Make sure the bed size doesn't decrease more than expected


In [ ]:
## Sanity Check - expected value 11966919
awk '{sum+=$3-$2} END {print sum}' \
    data/manually_created_files/cmrg_draft_benchmarks/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.01.bed 

In [ ]:
## Sanity Check - expected value 11966249
awk '{sum+=$3-$2} END {print sum}' \
    workflow/release_benchmark_generation/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.02.00.bed 

__Sanity Check__ Make sure the number of variants in the bed don't decrease more than expected

In [ ]:
## Sanity Check - expected value 205
intersectBed \
    -a data/manually_created_files/cmrg_draft_benchmarks/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.01.vcf.gz \
    -b data/manually_created_files/cmrg_draft_benchmarks/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.01.bed \
    | wc -l

In [ ]:
## Sanity check - expected value 204
intersectBed \
    -a data/manually_created_files/cmrg_draft_benchmarks/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.01.vcf.gz \
    -b workflow/release_benchmark_generation/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.02.00.bed \
    | wc -l


Using v0.01 variant calls as v0.02 variant calls

In [ ]:
cp data/manually_created_files/cmrg_draft_benchmarks/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.01.vcf.gz \
    workflow/release_benchmark_generation/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.02.00.vcf.gz 
cp data/manually_created_files/cmrg_draft_benchmarks/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.01.vcf.gz.tbi \
    workflow/release_benchmark_generation/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.02.00.vcf.gz.tbi 

In [ ]:
#remove ".;" at beginning of INFO field introduced by svanalyzer svwiden
gunzip -c workflow/release_benchmark_generation/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.02.00.vcf.gz \
    | sed 's/\.;REPTYPE/REPTYPE/' \
    | bgzip -c \
    > workflow/release_benchmark_generation/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v1.00.01.vcf.gz

tabix workflow/release_benchmark_generation/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v1.00.01.vcf.gz

In [ ]:
cp workflow/release_benchmark_generation/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v0.02.00.bed \
    workflow/release_benchmark_generation/HG002_GRCh37_difficult_medical_gene_SV_benchmark_v1.00.01.bed 

### Software versions

See `environment.yml` for dependencies. 
